In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import cv2
from torch.utils.data import DataLoader 

In [3]:
from retina.data.widerface import WiderFaceCustomDataset
from retina import transforms as CT
from torchvision import transforms as T

from torch.utils.data import DataLoader
from retina.data.widerface import detection_collate

from retina.ops.nms import py_cpu_nms
import numpy as np

from retina.models.retina import RetinaFace
from retina.data import config
from retina.modules.prior_box import PriorBox, CustomPriorBox
from retina.ops.boxes import decode, decode_landm
from retina.modules.loss import MultiBoxLoss

In [4]:
def adjust_learning_rate(optimizer, gamma, epoch, step_index, iteration, epoch_size):
    """Sets the learning rate
    # Adapted from PyTorch Imagenet example:
    # https://github.com/pytorch/examples/blob/master/imagenet/main.py
    """
    warmup_epoch = -1
    if epoch <= warmup_epoch:
        lr = 1e-6 + (initial_lr-1e-6) * iteration / (epoch_size * warmup_epoch)
    else:
        lr = initial_lr * (gamma ** (step_index))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
    return lr

In [5]:
BSIZE = 2
NUM_WORKERS = 0
LRATE = 1e-3
MOMENTUM = 0.9
WDECAY = 5e-4
GAMMA = 0.1
NUM_CLASSES = 2
IMG_SIZE = 640
RGB_MEANS = (104, 117, 123)

In [6]:
root = '/data/widerface/custom_dataset/train/'
dataset = WiderFaceCustomDataset(root=root, pair_transform=CT.Preprocess(img_dim=IMG_SIZE, rgb_means=RGB_MEANS))
loader = DataLoader(dataset, batch_size=BSIZE, shuffle=True, num_workers=NUM_WORKERS, collate_fn=detection_collate)

In [7]:
from retina.models.retina import RetinaFace, retina_face

model = retina_face(backbone="resnet", final_weight='../weights/Resnet50_Final.pth')
model.eval()
print()

Loading pretrained model from ../weights/Resnet50_Final.pth
remove prefix 'module.'
Missing keys:0
Unused checkpoint keys:0
Used keys:456



In [8]:
def get_prior_box_data(image_size, min_sizes=None, steps=None, clip=False, device="cpu"):
    img_width, img_height = image_size
    priorbox = CustomPriorBox(image_size=(img_width, img_height), min_sizes=min_sizes, steps=steps, clip=clip)
    with torch.no_grad():
        priors = priorbox.forward()
        priors = priors.to(device)
        prior_data = priors.data
    return prior_data

In [9]:
prior = get_prior_box_data(image_size=(IMG_SIZE,IMG_SIZE))
optimizer = optim.SGD(model.parameters(), lr=LRATE, momentum=MOMENTUM, weight_decay=WDECAY)
criterion = MultiBoxLoss(NUM_CLASSES, 0.35, True, 0, True, 7, 0.35, False)

In [10]:
# priorbox = CustomPriorBox(image_size=(IMG_SIZE, IMG_SIZE))
# with torch.no_grad():
#     priors = priorbox.forward()
#     priors = priors.cuda()




In [17]:
import torch
import torch.nn as nn
import torch.optim as optim

import pytorch_lightning as pl
from pytorch_lightning import loggers as pl_loggers


class RetinaFaceTask(pl.LightningModule):
    def __init__(self, model, optimizer, criterion, prior, scheduler=None):
        super(RetinaFaceTask, self).__init__()
        self.model = model
        self.criterion = criterion
        self.optimizer = optimizer
        self.prior = prior
        self.scheduler = scheduler
        # self.metric = Accuracy()
        
    def forward(self, images):
        output = self.model(images)
        return output
    
    def shared_step(self, batch, batch_idx):
        images, targets = batch
        images = images.to(self.device)
        targets = [anno.to(self.device) for anno in targets]
    
        predict = self.model.forward(images)
        
        loss_l, loss_c, loss_landm = self.criterion(predict, self.prior, targets)
        loc_weight = 2.0
        loss = loc_weight * loss_l + loss_c + loss_landm
        
        return loss
    
    def training_step(self, batch, batch_idx):
        loss = self.shared_step(batch, batch_idx)
        self.log('trn_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)

        return loss

    def validation_step(self, batch, batch_idx):
        loss = self.shared_step(batch, batch_idx)
        self.log('val_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)

        return loss

    def configure_optimizers(self):
        if self.scheduler:
            return [self.optimizer], [self.scheduler]
        return self.optimizer


In [18]:
retina_task = RetinaFaceTask(model=model, optimizer=optimizer, criterion=criterion, prior=prior)

In [19]:
import pytorch_lightning as pl
from retina.data.datamodule import WiderFaceDataModule

In [20]:
datamodule = WiderFaceDataModule(data_dir='/data/widerface/custom_dataset/')

In [21]:
datamodule.setup(stage="fit")

In [22]:
trainer = pl.Trainer(gpus=0)
trainer.fit(retina_task, datamodule)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name      | Type         | Params
-------------------------------------------
0 | model     | RetinaFace   | 27.3 M
1 | criterion | MultiBoxLoss | 0     
-------------------------------------------
27.3 M    Trainable params
0         Non-trainable params
27.3 M    Total params
109.174   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/opt/anaconda/envs/dlearn37/lib/python3.7/site-packages/pytorch_lightning/utilities/data.py:60: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 32. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  "Trying to infer the `batch_size` from an ambiguous collection. The batch size we"


Training: 0it [00:00, ?it/s]

/opt/anaconda/envs/dlearn37/lib/python3.7/site-packages/pytorch_lightning/trainer/trainer.py:688: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [ ]:
loader = datamodule.val_dataloader()
images, targets = next(iter(loader))

In [ ]:
trainset = datamodule.trainset
# images, targets = next(iter(loader))

In [ ]:
image, target = trainset[0]

In [ ]:
# target

In [ ]:
loader

In [ ]:
# targets

In [ ]:
len(trainset)